In [1]:
from scripts.import_all_data import ImportAllData

In [2]:
import_object = ImportAllData(
    configuration_file='/Users/timdunn/movie_project/all_processes/movie_configuration.yaml')

In [3]:
from scripts.data_to_db import *

In [4]:
from movies.models import *

In [5]:
from common_methods import *

In [ ]:
test_file = pd.read_csv('/Users/timdunn/test_imdb.csv')

In [ ]:
test_file.to_csv('/Users/timdunn/test_imdb.csv')

In [ ]:
test_file.loc[:, 'language'] = None

In [ ]:
data_object = DataToDb(Movie, '/Users/timdunn/test_imdb.csv')

In [ ]:
genres = Genre.objects.all()

In [ ]:
list(genres)

In [ ]:
data_object.import_records()

In [18]:
movie_title_sample = pd.read_csv('/Users/timdunn/Desktop/converted_movie_project_data/merged_imdb.csv')

In [19]:
movie_title_sample = movie_title_sample.loc[~movie_title_sample['tconst'].isin(tconsts)]

In [11]:
nonnull_air_dates = movie_title_sample.loc[~movie_title_sample['original_air_date'].isnull()]

In [ ]:
nonnull_air_dates

In [43]:
integer_columns = [column.attname for column in Movie._meta.get_fields() if
                           'IntegerField' in str(column.__class__)]

In [12]:
def not_null_value(value):
    return not pd.isnull(value) and value not in ['None', 'nan', None]

In [47]:
def value_converted(key, value):
    if key == 'is_adult' and value not in [None, 0, 1, True, False]:
        return False
    if key in integer_columns:
        try:
            return int(value)
        except:
            return 0
    return value

In [48]:
def compact_dict(row):
    initial_dict = {key: value for key, value in dict(row).items() if not_null_value(value)}
    initial_dict = {key: value_converted(key, value) for key, value in initial_dict.items()}
    return Movie(**assign_language(initial_dict, language_mapping))

In [49]:
def assign_language(the_dict, language_mapping):
    if the_dict.get('language') is not None:
        the_dict['language'] = language_mapping.get(the_dict.get('language'))
    return the_dict

In [50]:
movie_objects = list(movie_title_sample.apply(lambda row: compact_dict(row), axis=1))

In [35]:
np.unique(movie_title_sample['is_adult'])

array([0.000e+00, 1.000e+00, 1.981e+03, 2.005e+03, 2.014e+03, 2.017e+03,
       2.019e+03, 2.020e+03, 2.023e+03,       nan])

In [36]:
movie_title_sample

,tconst,title_type,primary_title,original_title,is_adult,start_year,runtime_minutes,end_year,original_air_date,language,...,score,poster,plot_outline,aspect_ratio,production_status,production_status_updated,bottom_100_rank,top_250_rank,box_office_budget,box_office_cumulative_worldwide_gross
100,tt0000102,short,The Beach at Villiers in a Gale,Plage de Villiers par gros temps,0.0,1896.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
101,tt0000103,short,Smarter than the Teacher,Plus fort que le maître,0.0,1896.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
102,tt0000104,short,Pompiers à Lyon,Pompiers à Lyon,0.0,1896.0,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
103,tt0000105,short,The Docks at Marseilles,Les quais à Marseille,0.0,1896.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
104,tt0000106,short,Retour au cantonnement,Retour au cantonnement,0.0,1896.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9711496,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0.0,2010.0,0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
9711497,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0.0,2010.0,0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
9711498,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0.0,2010.0,0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
9711499,tt9916856,short,The Wind,The Wind,0.0,2015.0,27,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [ ]:
dict(movie_objects[1])

In [22]:
language_mapping = Language.objects.distinct('name').in_bulk(field_name='name')

In [51]:
Movie.objects.bulk_create(movie_objects)

OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


In [ ]:
all_dicts = [dict(movie) for _, movie in movie_title_sample.iterrows()]

In [ ]:
compact_dict = {key: value for key, value in all_dicts[0].items() if not pd.isnull(value)}

In [ ]:
compact_dict

In [ ]:
compact_dict = [key for key in all_dicts[0].keys() if not pd.isnull(all_dicts[0][key])]

In [ ]:
compact_dict

In [ ]:
sample_dict = dict(movie_title_sample.loc[50])

In [ ]:
new_dict = dict()
for key in sample_dict.keys(): 
    if not pd.isnull(sample_dict[key]):
        new_dict[key] = sample_dict[key]

In [ ]:
Movie._meta.get_fields()

In [16]:
tconsts = Movie.objects.only('tconst')

In [17]:
tconsts = [tconst.tconst for tconst in list(tconsts)]

In [ ]:
tconsts

In [ ]:
from movies.models import Movie

In [ ]:
Movie.objects.bulk_create(**dict(new_dict))

In [ ]:
for key in enumerate(Language.objects.distinct('name').in_bulk(field_name='name')):
    print(key)
    

In [ ]:
Movie.objects.create(**dict(test_file.loc[0]))

In [ ]:
import_object.import_data()

In [ ]:
from common_methods import *

In [ ]:
aka_sample = pd.read_csv('/Users/timdunn/Desktop/converted_movie_project_data/aka.csv', nrows=100)

In [ ]:
aka_sample.loc[0:10]

In [ ]:
import_object.import_data()

In [ ]:
from movies.models import Movie

In [ ]:
Movie.objects.last()

In [ ]:
from scripts.convert_import_data import ConvertImportData

In [ ]:
convert_import_object = ConvertImportData(
    configuration_file='/Users/timdunn/movie_project/all_processes/movie_configuration.yaml')

In [ ]:
convert_import_object.import_data()

In [ ]:
class A:
    
    def __init__(self, class_object):
        self.awesome = 'yes'
        self.class_name = class_object.__name__

In [ ]:
class C:
    
    def __init__(self, class_object):
        self.awesome = 'no'
        self.class_name = class_object.__name__

In [ ]:
class B(A):
    
    def __init__(self, model, something='ok'):
        self.awesome = something
        super().__init__(model)

In [ ]:
B(A)

In [ ]:
from scripts.add_movie_data import *

In [ ]:
from all_processes.load_yaml_vars import LoadYamlVars

In [ ]:
from scripts.data_to_db import DataToDb

In [ ]:
from movies.models import Movie

In [ ]:
data_object = DataToDb(Movie, load_vars.merged_movie_title_path, [{"language": "name"}])

In [ ]:
load_vars = LoadYamlVars(yaml_file_path='/Users/timdunn/movie_project/all_processes/movie_configuration.yaml')

In [ ]:
add_title_object = AddMovieTitle(import_file_path=load_vars.merged_movie_title_path)

In [ ]:
from import_all_data import ImportAllData

In [ ]:
from add_movie_data import AddMovieTitle

In [ ]:
movie_title_object = AddMovieTitle(import_object.merged_movie_title_path)

In [ ]:
reflections = movie_title_object.movie_options().get('reflections')

In [ ]:
movie_title_object.movie_options().keys()

In [ ]:
movie_title_object.foreign_model_reflections()

In [ ]:
reflections

In [ ]:
import_object.merged_movie_title_path

In [ ]:
AddMovieTitle

In [ ]:
import_object = ImportAllData(configuration_file='/Users/timdunn/movie_project/all_processes/movie_configuration.yaml')

In [ ]:
add_movie_title_object = AddMovieTitle(import_file_path=import_object.import_class_dict().get(AddMovieTitle))

In [ ]:
ImportAllData

In [ ]:
convert_import_object.convert_and_import()